# ML-6 : ONNX Model Integration avec ML.NET

**Navigation** : [Index](README.md) | [<< ML-5-TimeSeries](ML-5-TimeSeries.ipynb) | [Suivant >>](ML-7-Recommendation.ipynb)

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Comprendre ce qu'est **ONNX** (Open Neural Network Exchange)
2. Charger des modèles ONNX externes dans ML.NET
3. Exporter des modèles ML.NET vers ONNX
4. Utiliser des modèles Python (Scikit-learn, PyTorch) dans des applications .NET
5. Optimiser les performances avec ONNX Runtime

### Prérequis
- ML-1 à ML-5 complétés
- Connaissance de base de Python et Scikit-learn

### Durée estimée : 45-60 minutes

---

# Introduction à ONNX

## Qu'est-ce qu'ONNX ?

**ONNX (Open Neural Network Exchange)** est un format ouvert pour représenter des modèles de machine learning.

**Avantages** :
- **Interopérabilité** : Échanger des modèles entre frameworks (PyTorch ↔ TensorFlow ↔ Scikit-learn ↔ ML.NET)
- **Performance** : ONNX Runtime optimise l'inférence
- **Portabilité** : Déployer sur différentes plateformes (Cloud, Edge, Mobile)

**Écosystème ONNX** :
```
Python (PyTorch/Scikit-learn) → ONNX → ML.NET → Application .NET
                        ↓
                    ONNX Runtime (optimisation)
```

## Scénarios d'utilisation

| Scénario | Avantage |
|----------|----------|
| Data science en Python, production en .NET | Utiliser les outils Python pour R&D, déployer en .NET |
| Modèles pré-entraînés Hugging Face | Accès à des milliers de modèles SOTA |
| Multi-framework | Combiner des modèles de différentes sources |
| Edge computing | ONNX Runtime optimisé pour CPU/GPU |

In [1]:
#r "nuget: Microsoft.ML, 3.0.0"
#r "nuget: Microsoft.ML.OnnxTransformer, 3.0.0"
#r "nuget: Microsoft.ML.OnnxRuntime, 1.16.0"

using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms.Onnx;
using System;
using System.Collections.Generic;
using System.Linq;

Console.WriteLine("Packages ONNX chargés avec succès !");

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.ML, 3.0.0 Microsoft.ML.OnnxRuntime, 1.16.0 Microsoft.ML.OnnxTransformer, 3.0.0

Packages ONNX chargés avec succès !


## Exemple 1 : Charger un modèle ONNX externe

Dans cet exemple, nous allons charger un modèle ONNX dans ML.NET.

> **Note** : Pour ce notebook, nous allons créer un modèle ONNX simple. En pratique, vous utiliseriez des modèles exportés depuis PyTorch, TensorFlow ou Scikit-learn.

In [2]:
// Créer le contexte ML
var mlContext = new MLContext(seed: 42);

// Définir les classes de données
public class OnnxInputData
{
    [VectorType(4)]
    public float[] Features { get; set; }
}

public class OnnxOutputData
{
    public float Prediction { get; set; }
    public float[] Probability { get; set; }
}

// Exemple de données
var sampleData = new OnnxInputData
{
    Features = new float[] { 5.1f, 3.5f, 1.4f, 0.2f }
};

Console.WriteLine("Données d'exemple créées");
Console.WriteLine($"Features: [{string.Join(", ", sampleData.Features)}]");

Données d'exemple créées


Features: [5,1, 3,5, 1,4, 0,2]


### Chargement d'un modèle ONNX

Dans un scénario réel, vous auriez un fichier `.onnx` exporté depuis Python.

**Exemple d'export depuis Python** :
```python
# Entrainement avec Scikit-learn
from sklearn.ensemble import RandomForestClassifier
from skl2onnx import convert_sklearn

model = RandomForestClassifier()
model.fit(X_train, y_train)

# Export vers ONNX
onnx_model = convert_sklearn(model, name='model')
with open('model.onnx', 'wb') as f:
    f.write(onnx_model.SerializeToString())
```

In [3]:
// NOTE: Ce code montre comment charger un modèle ONNX.
// En pratique, remplacez 'model.onnx' par votre fichier.

/*
// Chargement du modèle ONNX
var onnxModelPath = "model.onnx";

var pipeline = mlContext.Transforms
    .ApplyOnnxModel(
        modelFile: onnxModelPath,
        outputColumnNames: new[] { "Prediction", "Probability" },
        inputColumnNames: new[] { "Features" }
    );

// Créer le moteur de prédiction
var predictionEngine = mlContext.Model.CreatePredictionEngine<OnnxInputData, OnnxOutputData>(pipeline);

// Faire une prédiction
var prediction = predictionEngine.Predict(sampleData);

Console.WriteLine($"Prediction: {prediction.Prediction}");
Console.WriteLine($"Probabilities: [{string.Join(", ", prediction.Probability)}]");
*/

Console.WriteLine("Pour utiliser ce code:");
Console.WriteLine("1. Exportez un modèle depuis Python (skl2onnx, torch.onnx)");
Console.WriteLine("2. Placez le fichier .onnx dans le répertoire du notebook");
Console.WriteLine("3. Décommentez le code ci-dessus et exécutez");

Pour utiliser ce code:


1. Exportez un modèle depuis Python (skl2onnx, torch.onnx)


2. Placez le fichier .onnx dans le répertoire du notebook


3. Décommentez le code ci-dessus et exécutez


## Exemple 2 : Exporter un modèle ML.NET vers ONNX

ML.NET permet également d'**exporter** des modèles entraînés vers ONNX pour les utiliser dans d'autres frameworks.

In [4]:
// Entraîner un modèle simple avec ML.NET
public class TrainingData
{
    public float Feature1 { get; set; }
    public float Feature2 { get; set; }
    public bool Label { get; set; }
}

// Générer des données d'entraînement
var trainingData = new List<TrainingData>();
var rand = new Random(42);

for (int i = 0; i < 100; i++)
{
    trainingData.Add(new TrainingData
    {
        Feature1 = (float)rand.NextDouble(),
        Feature2 = (float)rand.NextDouble(),
        Label = rand.NextDouble() > 0.5
    });
}

var dataView = mlContext.Data.LoadFromEnumerable(trainingData);

// Créer et entraîner un pipeline
var pipeline = mlContext.Transforms.Concatenate("Features", "Feature1", "Feature2")
    .Append(mlContext.BinaryClassification.Trainers.SdcaLogisticRegression(
        labelColumnName: "Label",
        featureColumnName: "Features"
    ));

var model = pipeline.Fit(dataView);

Console.WriteLine("Modèle ML.NET entraîné avec succès !");

Modèle ML.NET entraîné avec succès !


In [5]:
// Exporter le modèle vers ONNX
/*
using Microsoft.ML.Onnx;

var onnxExportPath = "model_exported.onnx";

// Convertir le modèle ML.NET en ONNX
mlContext.Model.ConvertToOnnx(
    model: model,
    input: dataView,
    outputFilePath: onnxExportPath
);

Console.WriteLine($"Modèle exporté vers {onnxExportPath}");

// Le modèle ONNX peut maintenant être utilisé dans :
// - Python (onnxruntime)
// - JavaScript (onnxruntime-web)
// - C++ (onnxruntime)
// - Java (onnxruntime)
*/

Console.WriteLine("NOTE: L'export ONNX dans ML.NET est expérimental.");
Console.WriteLine("Consultez la documentation pour les modèles supportés.");

NOTE: L'export ONNX dans ML.NET est expérimental.


Consultez la documentation pour les modèles supportés.


## Exemple 3 : Utiliser des modèles Hugging Face avec ONNX

**Hugging Face** propose des milliers de modèles pré-entraînels compatibles ONNX.

**Modèles populaires** :
- **BERT** : Classification de texte, NER, Question-Answering
- **GPT** : Génération de texte
- **Whisper** : Reconnaissance vocale
- **ViT** : Classification d'images

In [6]:
// Exemple conceptuel d'utilisation d'un modèle BERT ONNX

/*
// 1. Télécharger le modèle BERT depuis Hugging Face
// python -m transformers bert-base-uncased --onnx

// 2. Charger dans ML.NET
public class BertInput
{
    [VectorType(512)]
    public long[] InputIds { get; set; }
    
    [VectorType(512)]
    public long[] AttentionMask { get; set; }
    
    [VectorType(512)]
    public long[] TokenTypeIds { get; set; }
}

public class BertOutput
{
    [VectorType(2)]
    public float[] Output { get; set; }  // Probabilités de classification
}

var bertPipeline = mlContext.Transforms.ApplyOnnxModel(
    modelFile: "bert-base-uncased.onnx",
    outputColumnNames: new[] { "output" },
    inputColumnNames: new[] { "input_ids", "attention_mask", "token_type_ids" }
);
*/

Console.WriteLine("Modèles Hugging Face supportés:");
Console.WriteLine("- bert-base-uncased (classification)");
Console.WriteLine("- distilbert-base-uncased (classification rapide)");
Console.WriteLine("- whisper-small (reconnaissance vocale)");
Console.WriteLine("\nPour convertir: transformers-cli export --model <nom> --onnx");

Modèles Hugging Face supportés:


- bert-base-uncased (classification)


- distilbert-base-uncased (classification rapide)


- whisper-small (reconnaissance vocale)



Pour convertir: transformers-cli export --model <nom> --onnx


## Exemple 4 : Workflow complet Python → ONNX → ML.NET

Voici un workflow typique de Data Science en Python avec déploiement en .NET :

### Étape 1 : Python (R&D)
```python
# research_model.py
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from skl2onnx import convert_sklearn

# Charger les données
df = pd.read_csv('data.csv')
X, y = df.drop('target', axis=1), df['target']

# Entraîner le modèle
model = GradientBoostingClassifier(n_estimators=100)
model.fit(X, y)

# Exporter vers ONNX
onnx_model = convert_sklearn(
    model,
    initial_types=[('input', FloatTensorType([None, X.shape[1]]))]
)

with open('gradient_boosting_model.onnx', 'wb') as f:
    f.write(onnx_model.SerializeToString())
```

### Étape 2 : .NET (Production)
```csharp
using Microsoft.ML;
using Microsoft.ML.Data;

var mlContext = new MLContext();

// Charger le modèle ONNX
var pipeline = mlContext.Transforms.ApplyOnnxModel(
    modelFile: "gradient_boosting_model.onnx",
    outputColumnNames: new[] { "output_label" },
    inputColumnNames: new[] { "input" }
);

// Créer l'API de prédiction
var engine = mlContext.Model.CreatePredictionEngine<Input, Output>(pipeline);
```

**Avantages** :
- R&D rapide avec Python (pandas, scikit-learn)
- Déploiement robuste avec .NET (ASP.NET, Azure Functions)
- Performance optimisée avec ONNX Runtime

## Optimisation des performances

### ONNX Runtime

**ONNX Runtime** est un moteur d'inférence haute performance :

| Optimisation | Impact |
|--------------|--------|
| **Graph optimization** | Fusion des opérations, élimination des nœuds inutiles |
| **Quantization** | INT8 au lieu de FP32 (4x moins de mémoire) |
| **Parallel execution** | Multi-threading automatique |
| **GPU acceleration** | CUDA, TensorRT, OpenVINO |

### Comparaison des performances

| Framework | Latence (ms) | Throughput (req/s) |
|-----------|--------------|-------------------|
| Scikit-learn (Python) | 15 | 66 |
| PyTorch (CPU) | 10 | 100 |
| ONNX Runtime (CPU) | 5 | 200 |
| ONNX Runtime (GPU) | 1 | 1000 |

## Résumé et conclusion

Ce notebook a présenté l'intégration **ONNX** dans ML.NET :

| Concept | Clé |
|---------|-----|
| **ONNX** | Format ouvert pour échanger des modèles entre frameworks |
| **ApplyOnnxModel** | Transformer ML.NET pour charger des modèles ONNX |
| **skl2onnx** | Outil Python pour exporter Scikit-learn vers ONNX |
| **torch.onnx** | Export PyTorch vers ONNX |
| **ONNX Runtime** | Moteur d'inférence haute performance |

**Workflow recommandé** :
```
1. R&D en Python (pandas, scikit-learn, PyTorch)
2. Export vers ONNX (skl2onnx, torch.onnx)
3. Import dans ML.NET (ApplyOnnxModel)
4. Déploiement .NET (ASP.NET, Azure Functions)
```

**Points clés** :
1. ONNX permet d'utiliser des modèles Python dans des applications .NET
2. ONNX Runtime optimise automatiquement les performances
3. Supporte CPU, GPU et divers accélérateurs matériels
4. Hugging Face propose des milliers de modèles ONNX pré-entraînés

**Limitations** :
- Tous les opérateurs ML.NET ne sont pas exportables vers ONNX
- Les modèles personnalisés nécessitent une conversion manuelle
- La compatibilité dépend des versions ONNX

**Pour aller plus loin** :
- [ONNX Runtime Documentation](https://onnxruntime.ai/docs/)
- [Hugging Face ONNX Hub](https://huggingface.co/models?library=onnx)
- [skl2onnx Tutorial](https://onnx.ai/sklearn-onnx/)

---

**Navigation** : [<< ML-5-TimeSeries](ML-5-TimeSeries.ipynb) | [Suivant >> ML-7-Recommendation](ML-7-Recommendation.ipynb)